In [1]:
import subprocess
import sys

COLAB = "google.colab" in sys.modules


def _install(package):
    if COLAB:
        ans = input(f"Install { package }? [y/n]:")
        if ans.lower() in ["y", "yes"]:
            subprocess.check_call(
                [sys.executable, "-m", "pip", "install", "--quiet", package]
            )
            print(f"{ package } installed!")


def _colab_install_missing_deps(deps):
    import importlib

    for dep in deps:
        if importlib.util.find_spec(dep) is None:
            if dep == "iris":
                dep = "scitools-iris"
            _install(dep)


deps = ["ckanapi", "geopandas"]


_colab_install_missing_deps(deps)

# Programmatically query the IOOS Data Catalog for a specific observation type

Created: 2024-09-17

Updated: 2025-03-06

Author: [Mathew Biddle](mailto:mathew.biddle@noaa.gov)

In this notebook we highlight the ability to search the [IOOS Data Catalog](https://data.ioos.us/) for a specific subset of observations using the [CKAN](https://ckan.org/) web accessible Application Programming Interface (API). 

For this example, we want to look for observations of oxygen in the water column across the IOOS Catalog. As part of the [IOOS Metadata Profile](https://ioos.github.io/ioos-metadata/), which the US IOOS community uses to publish datasets, we know that each Regional Association and DAC will be following the [Climate and Forecast (CF) Conventions](http://cfconventions.org/) and using CF `standard_names` to describe their datasets. So, with that assumption, we can search across the IOOS Data catalog for datasets with the CF standard names that contain `oxygen` and `sea_water`. Then, we can build a simple map to show the geographical distribution of those datasets.

## Build CKAN API query base.

Uses https://github.com/ckan/ckanapi

In [2]:
from ckanapi import RemoteCKAN

ua = "ckanapiioos/1.0 (+https://ioos.us/)"

# ioos_catalog = RemoteCKAN('https://data.ioos.us', user_agent=ua, get_only=True)
ioos_catalog = RemoteCKAN("https://data.ioos.us", user_agent=ua)
ioos_catalog

## What organizations are in the catalog?

Tell me what organizations are there.

In [3]:
orgs = ioos_catalog.action.organization_list()
print(orgs)

['aoos', 'caricoos', 'cdip', 'cencoos', 'comt', 'gcoos', 'glider-dac', 'glos', 'hf-radar-dac', 'ioos', 'maracoos', 'nanoos', 'neracoos', 'noaa-co-ops', 'noaa-ndbc', 'oceansites', 'pacioos', 'sccoos', 'secoora', 'unidata', 'usgs', 'us-navy']


## How many datasets are we searching across?

Grab all the datasets available and return the count.

In [4]:
# datasets = ioos_catalog.action.package_search(fq='+cf_standard_names:mass_concentration_of_oxygen_in_sea_water', rows=50)
datasets = ioos_catalog.action.package_search()
datasets["count"]

36168

## Grab the most recent applicable CF standard names

Collect [CF standard names](https://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html) that contain `oxygen` and `sea_water` from the CF standard name list.

In [5]:
import pandas as pd

url = "https://cfconventions.org/Data/cf-standard-names/current/src/cf-standard-name-table.xml"

tbl_version = pd.read_xml(url, xpath="./*")["version_number"][0].astype(int)
df = pd.read_xml(url, xpath="entry")

std_names = df.loc[
    (df["id"].str.contains("oxygen") & df["id"].str.contains("sea_water"))
]

print(f"CF Standard Name Table: {tbl_version}")

std_names[["id", "description"]]

CF Standard Name Table: 89


,id,description
470,depth_at_shallowest_local_minimum_in_vertical_...,Depth is the vertical distance below the surfa...
626,fractional_saturation_of_oxygen_in_sea_water,Fractional saturation is the ratio of some mea...
1361,mass_concentration_of_oxygen_in_sea_water,Mass concentration means mass per unit volume ...
1729,mole_concentration_of_dissolved_molecular_oxyg...,Mole concentration means number of moles per u...
1730,mole_concentration_of_dissolved_molecular_oxyg...,"""Mole concentration at saturation"" means the m..."
1731,mole_concentration_of_dissolved_molecular_oxyg...,Mole concentration means number of moles per u...
1829,mole_concentration_of_preformed_dissolved_mole...,"""Mole concentration"" means the number of moles..."
2000,moles_of_oxygen_per_unit_mass_in_sea_water,"moles_of_X_per_unit_mass_inY is also called ""m..."
3217,surface_molecular_oxygen_partial_pressure_diff...,"The surface called ""surface"" means the lower b..."
3717,temperature_of_sensor_for_oxygen_in_sea_water,Temperature_of_sensor_for_oxygen_in_sea_water ...


## Search across IOOS Data Catalog using CKAN API

Search the IOOS Data Catalog for CF standard names that match those above.

In [6]:
import time

from ckanapi import RemoteCKAN
from ckanapi.errors import CKANAPIError

ua = "ckanapiioos/1.0 (+https://ioos.us/)"

ioos_catalog = RemoteCKAN("https://data.ioos.us", user_agent=ua)
ioos_catalog

df_out = pd.DataFrame()

for std_name in std_names["id"]:

    print(std_name)

    fq = f"+cf_standard_names:{std_name}"

    result_count = 0

    df_std_name = pd.DataFrame()
    
    while True:
        try:
            datasets = ioos_catalog.action.package_search(fq=fq, rows=500, start=result_count)
        except CKANAPIError:
            continue

        num_results = datasets["count"]

        print(f"num_results: {num_results}, result_count: {result_count}")

        for dataset in datasets["results"]:
            # print(dataset['title'])
            df = pd.DataFrame(
                {
                    "title": [dataset["title"]],
                    "url": [dataset["resources"][0]["url"]],
                    "org": [dataset["organization"]["title"]],
                    "std_name": std_name,
                }
            )

            df_std_name = pd.concat([df_std_name, df], ignore_index=True)
            result_count = df_std_name.shape[0]

        time.sleep(1)

        if result_count >= num_results:
            print(f"num_results: {num_results}, result_count: {result_count}")
            break
            
    df_out = pd.concat([df_out, df_std_name], ignore_index=True)
    
    print(f"num_results: {num_results}, result_count: {result_count}, total_result_count: {df_out.shape[0]}")
    
df_out.shape

depth_at_shallowest_local_minimum_in_vertical_profile_of_mole_concentration_of_dissolved_molecular_oxygen_in_sea_water
num_results: 0, result_count: 0
num_results: 0, result_count: 0
fractional_saturation_of_oxygen_in_sea_water
num_results: 5070, result_count: 0
num_results: 5070, result_count: 500
num_results: 5070, result_count: 1000
num_results: 5070, result_count: 1500
num_results: 5070, result_count: 2000
num_results: 5070, result_count: 2500
num_results: 5070, result_count: 3000
num_results: 5070, result_count: 3500
num_results: 5070, result_count: 4000
num_results: 5070, result_count: 4500
num_results: 5070, result_count: 5000
num_results: 5070, result_count: 5070
mass_concentration_of_oxygen_in_sea_water
num_results: 4052, result_count: 0
num_results: 4052, result_count: 500
num_results: 4052, result_count: 1000
num_results: 4052, result_count: 1500
num_results: 4052, result_count: 2000
num_results: 4052, result_count: 2500
num_results: 4052, result_count: 3000
num_results: 405

(12622, 4)

## Do some summarizing of the responses

The DataFrame of the matching datasets is quite large. I wonder what the distribution of those datasets across organizations looks like? Let's use [pandas.groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) to generate some statistics about how many datasets are provided, matching our criteria, by which organization.

In [7]:
df_out.groupby(by="org").count()

,title,url,std_name
org,,,
AOOS,28,28,28
CeNCOOS,414,414,414
GCOOS,9547,9547,9547
Glider DAC,2204,2204,2204
MARACOOS,183,183,183
NANOOS,4,4,4
NERACOOS,48,48,48
PacIOOS,16,16,16
SCCOOS,1,1,1


## Drop the Glider DAC data

Glider DAC data are already making it to NCEI, so we can drop those entries.

In [8]:
df_out_no_glider = df_out.loc[~df_out["org"].str.contains("Glider DAC")]
df_out_no_glider.groupby(by="org").count()

,title,url,std_name
org,,,
AOOS,28,28,28
CeNCOOS,414,414,414
GCOOS,9547,9547,9547
MARACOOS,183,183,183
NANOOS,4,4,4
NERACOOS,48,48,48
PacIOOS,16,16,16
SCCOOS,1,1,1
SECOORA,177,177,177


## Digging into some of the nuances

There are still quite a lot of datasets from each organization. As our search above looked for each CF standard_name across all the datasets, there might be duplicate datasets which have multiple matching CF standard names. ie. one dataset might have both `mass_concentration_of_oxygen_in_sea_water` and `fractional_saturation_of_oxygen_in_sea_water`, but we only need to know that it's one dataset.

As we only need to know about the unique datasets, let's count how many unique dataset urls we have.

In [9]:
df_out_no_glider.groupby(by="url").count()

,title,org,std_name
url,,,
http://www.humboldt.edu,4,4,4
http://www.neracoos.org/erddap/tabledap/A01_aanderaa_o2_all,1,1,1
http://www.neracoos.org/erddap/tabledap/A01_optode_all,2,2,2
http://www.neracoos.org/erddap/tabledap/A01_sbe16_disox_all,2,2,2
http://www.neracoos.org/erddap/tabledap/C02_sbe16_disox_all,2,2,2
...,...,...,...
https://pae-paha.pacioos.hawaii.edu/erddap/tabledap/nss_012,2,2,2
https://pae-paha.pacioos.hawaii.edu/erddap/tabledap/nss_013,2,2,2
https://pae-paha.pacioos.hawaii.edu/erddap/tabledap/wqb_04,2,2,2


## Drop duplicate records

As you can see above, there are a lot of duplicate dataset urls which we can simplify down. We identify duplicates by looking at the URL, which should be unique for each dataset, and drop the duplicates.

In [10]:
df_out_nodups_no_glider = df_out_no_glider.drop_duplicates(subset=["url"], keep="last")

df_out_nodups_no_glider

,title,url,org,std_name
856,"Great Bay,NH. Great Bay WQ station",http://www.neracoos.org/erddap/tabledap/GRBGBW...,NERACOOS,fractional_saturation_of_oxygen_in_sea_water
857,"Great Bay,NH. Oyster River WQ station",http://www.neracoos.org/erddap/tabledap/GRBORW...,NERACOOS,fractional_saturation_of_oxygen_in_sea_water
863,"Great Bay,NH. Squamscott River WQ station",http://www.neracoos.org/erddap/tabledap/GRBSQW...,NERACOOS,fractional_saturation_of_oxygen_in_sea_water
867,"Great Bay,NH. Lamprey River WQ station",http://www.neracoos.org/erddap/tabledap/GRBLRW...,NERACOOS,fractional_saturation_of_oxygen_in_sea_water
989,Monterey Bay Aquarium Seawater Intake,https://erddap.cencoos.org/erddap/tabledap/mon...,CeNCOOS,fractional_saturation_of_oxygen_in_sea_water
...,...,...,...,...
12617,"Walton-Smith CTD, WS1102, WS1102_028, 2011-02-...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS,volume_fraction_of_oxygen_in_sea_water
12618,"Walton-Smith CTD, WS0718, WS0718_WS0718_056, 2...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS,volume_fraction_of_oxygen_in_sea_water
12619,"Walton-Smith CTD, WS0802, WS0802_WS0802_Wet_40...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS,volume_fraction_of_oxygen_in_sea_water
12620,"Walton-Smith CTD, WS20342, WS20342_WS20342_stn...",https://gcoos5.geos.tamu.edu/erddap/tabledap/W...,GCOOS,volume_fraction_of_oxygen_in_sea_water


## How many endpoints are not ERDDAP?

Now we have a unique list of datasets which match our CF standard name criteria. Since we have some background in using [ERDDAP to query for data](https://ioos.github.io/ioos_code_lab/content/code_gallery/data_access_notebooks/2017-03-21-ERDDAP_IOOS_Sensor_Map.html), let's take a look at what other endpoints each of the datasets are using.

_Hint: We know ERDDAP systems typically have `erddap` in their urls._

In [11]:
df_out_nodups_no_glider.loc[~df_out_nodups_no_glider["url"].str.contains("erddap")]

,title,url,org,std_name
9001,CeNCOOS in situ water monitoring data at Trini...,http://www.humboldt.edu,CeNCOOS,mass_concentration_of_oxygen_in_sea_water
9081,"MAPCO2 Buoy: Maihi Bay, Hawaii Island, Hawaii",https://www.nodc.noaa.gov/ocads/oceans/time_se...,PacIOOS,mass_concentration_of_oxygen_in_sea_water


## What's the remaining distribution?

This is the distribution of unique datasets found in the IOOS Data Catalog which have a CF Standard Name that contains the work `oxygen` and `sea_water`. We've dropped out the Glider DAC datasets as, theoretically, those are in NCEI already.

In [12]:
df_out_nodups_no_glider.groupby(by="org").count()

,title,url,std_name
org,,,
AOOS,13,13,13
CeNCOOS,215,215,215
GCOOS,7152,7152,7152
MARACOOS,139,139,139
NANOOS,2,2,2
NERACOOS,32,32,32
PacIOOS,7,7,7
SCCOOS,1,1,1
SECOORA,93,93,93


## Ingest data

Let's rip through all of the datasets, grab the data as a table (including units) and make a monster dictionary. This takes a bit.

In [13]:
dict_out_final = {}

for index, row in df_out_nodups_no_glider.iterrows():
    dict_out_final["{}".format(row["title"])] = pd.read_csv(
        "{}.csvp".format(row["url"]), low_memory=False
    )

HTTPError: HTTP Error 502: Bad Gateway

Let's take a quick look at one of the DataFrames.

Transpose it when we print, so we can see all the columns.

In [ ]:
dict_out_final['"Deepwater CTD - pe972218.ctd.nc - 29.25N, -87.89W - 1997-03-21"'].head(
    5
).T

## Let's make a nice map of the distribution of observations

Below we create a mapping function to plot the unique dataset points on a map. Then, we use that function with our full response. We have to do a little reorganizing of the data to build one DataFrame for all the coordinates.

In [ ]:
import cartopy.io.shapereader as shpreader
import geopandas as gpd
import matplotlib.pyplot as plt


def make_map(df):
    # initialize an axis
    fig, ax = plt.subplots(figsize=(8, 6))  # plot map on axis
    shpfilename = shpreader.natural_earth(
        resolution="110m",
        category="cultural",
        name="admin_0_countries",
    )
    countries = gpd.read_file(shpfilename)

    countries[countries["NAME"] == "United States of America"].plot(
        color="lightgrey", ax=ax
    )

    # plot points
    df.plot(
        x="longitude (degrees_east)",
        y="latitude (degrees_north)",
        kind="scatter",
        ax=ax,
    )  # add grid

    ax.grid(visible=True, alpha=0.5)

    return ax

In [ ]:
df_coords = pd.DataFrame(
    columns=["latitude (degrees_north)", "longitude (degrees_east)"]
)

for key in dict_out_final.keys():
    df_coords = pd.concat(
        [
            df_coords,
            dict_out_final[key][
                ["latitude (degrees_north)", "longitude (degrees_east)"]
            ],
        ]
    )

# drop all duplicates
df_coords_clean = df_coords.drop_duplicates(ignore_index=True)

# make the map
make_map(df_coords_clean)

## Lets explore those points on an interactive map

Just for fun, we can us [`geopandas.explore()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html) to plot these points on an interactive map to browse around.

In [ ]:
gdf = gpd.GeoDataFrame(
    df_coords_clean,
    geometry=gpd.points_from_xy(
        df_coords_clean["longitude (degrees_east)"],
        df_coords_clean["latitude (degrees_north)"],
    ),
    crs="EPSG:4326",
)

gdf.explore()

We hope this example demonstrates the flexibility of direct requests to the IOOS Data Catalog CKAN server and all the possibilities it provides. In this notebook we:

* Search the IOOS Data Catalog CKAN API with keywords.
* Found datasets matching our specified criteria.
* Collected all the data from each of the datasets matching our criteria.
* Created a simple map of the distribution of datasets which match our criteria.

To take this one step further, since we collected all the data from each of the datasets (in the dictionary `dict_out_final`) a user could integrate all of the oxygen observations together and start to build a comprehensive dataset. 

Additionally, a user could modify the CKAN query to search for terms outside of the CF standard names to potentially gather more datasets. 